In [16]:
import sys
import os

%load_ext autoreload
%autoreload 2

# Usar el directorio de trabajo actual en lugar de __file__
path = os.getcwd()
sys.path.insert(0, os.path.join(path, '..', 'src'))
import joblib
import pandas as pd

import data_preprocesing as dp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<unknown>:11: SyntaxWarning: invalid escape sequence '\p'
<unknown>:11: SyntaxWarning: invalid escape sequence '\p'


In [22]:

  # Ajusta si cambia la ruta

# 1. Cargar base de validación
df_val = pd.read_csv(r'..\data\raw\base_validacion.csv', sep='|', encoding='utf-8')

# 2. Preprocesamiento sin limpieza ni balanceo
preprocesor = dp.ModelPreprocessor(apply_cleaning=True)
X_val, _ = preprocesor.transform(df_val)

# 3. Cargar modelo
model = joblib.load(r'..\models\model.pkl')

# 4. Predecir
y_pred_proba = model.predict_proba(X_val)[:, 1]

# 5. Función de clasificación de grupo de riesgo
def asignar_grupo(prob):
    if 0 <= prob <= 0.01:
        return 't1'
    elif 0.01 < prob <= 0.015:
        return 't2'
    elif 0.015 < prob <= 0.03:
        return 't3'
    elif 0.03 < prob <= 0.045:
        return 't4'
    elif 0.045 < prob <= 0.08:
        return 't5'
    elif 0.08 < prob <= 0.15:
        return 't6'
    elif 0.15 < prob <= 0.30:
        return 't7'
    elif 0.30 < prob <= 1.0:
        return 't8'
    else:
        return 'fuera_rango'

# 6. Construir DataFrame de respuesta alineado al índice procesado
df_respuesta = pd.DataFrame({
    'num_doc': df_val.loc[X_val.index, 'num_doc'].values,
    'probabilidad': y_pred_proba
})
df_respuesta['grupo_riesgo'] = df_respuesta['probabilidad'].apply(asignar_grupo)

# 7. Validación
assert df_respuesta['grupo_riesgo'].isin(['t1','t2','t3','t4','t5','t6','t7','t8']).all(), "❌ Hay categorías fuera de rango"

# 8. Guardar CSV
# df_respuesta.to_csv('../entregables/base_prueba_calificada.csv', index=False)


In [24]:
df_respuesta.groupby('grupo_riesgo').size().to_frame('cantidad').reset_index().sort_values(by='grupo_riesgo')

,grupo_riesgo,cantidad
0,t4,2
1,t5,20
2,t6,39
3,t7,98
4,t8,441
